In [6]:
import openai
import requests
import base64
import os
from google.colab import files

def upload_and_get_path():
  uploaded = files.upload()
  for filename in uploaded.keys():
    print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')
    return filename
  return None


# OpenAI API 키 설정
API_KEY = "YOUR_API_KEY"

# 이미지 파일을 Base64로 변환하는 함수
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# GPT-4 Vision을 활용한 수학 문제 풀이 함수
def solve_math_problem_with_image(image_path, problem_text):
    encoded_image = encode_image(image_path)

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": "gpt-4o-mini",
        "messages": [
            {"role": "system", "content": """
            문제의 난이도를 구분해주는 챗봇입니다.

            난이도 구분은 이러합니다

            10등급: 간단한 계산 등, 암산으로도 풀 수 있는 문제
            9등급: 어느 정도의 아이디어가 약간은 필요하지만, 아이디어만 있다면 암산으로도 풀리는 문제
            8등급: 아이디어가 필요하고, 약간의 계산 과정을 필요로 하는 문제
            7등급: 아이디어가 2가지 이상 필요하고, 계산이 비교적 간단한 문제
            6등급: 아이디어가 2가지 이상 필요하고, 계산이 7등급에 비해 더 필요한 문제
            5등급: 아이디어가 필요하고, 계산이 복잡하며, 경우를 나눠야 되는 경우가 있는 문제
            4등급: 5등급에 비해 계산이 훨씬 복잡하고, 아이디어가 3개 이상 필요한 문제
            3등급: 고등학교 과정에서 요구되는 아이디어가 5개 이상 필요한 문제
            2등급: 고등학교 과정에서 요구되는 아이디어가 6개 이상 필요하고, 3등급에 비해 계산이 더 요구되는 문제
            1등급: 계산이 가장 복잡하고, 아이디어도 가장 많이 필요한 문제

            답변의 형식은 이러합니다:

            문제의 난이도 <>등급

            문제에 필요한 개념과 아이디어: 종류와 설명

            문제의 난이도가 이러한 이유: 설명

            <>에 들어갈 숫자는 assistant가 판단합니다
            """},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": problem_text},
                    {"type": "image_url", "image_url": {"url" : f"data:image/png;base64,{encoded_image}"}}
                ],
            },
        ],
        "temperature" : 0,
        "max_tokens": 500
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", json=payload, headers=headers)

    return response.json()

# 실행 예제
image_path = upload_and_get_path()  # 수학 문제 이미지 파일 경로
problem_text = "이 문제의 난이도를 알려주고, 왜 그러한 난이도를 가지고 있는지 알려줘"

response = solve_math_problem_with_image(image_path, problem_text)

# 결과 출력
print(response["choices"][0]["message"]["content"].replace("\\n", "<br>"))


Saving tngkr1.png to tngkr1.png
User uploaded file "tngkr1.png" with length 75409 bytes
문제의 난이도 4등급

문제에 필요한 개념과 아이디어: 
1. 수열의 정의: 주어진 수열의 규칙을 이해하고 적용하는 능력.
2. 경우의 수: 주어진 조건에 따라 수열의 항을 나누어 계산하는 방법.
3. 수열의 수렴성: 수열이 어떤 값으로 수렴하는지 분석하는 능력.

문제의 난이도가 이러한 이유: 
이 문제는 수열의 정의와 규칙을 이해하고, 주어진 조건에 따라 여러 경우를 나누어 계산해야 하므로, 기본적인 수학적 사고와 계산이 필요합니다. 또한, 수열의 수렴성을 분석하는 과정이 포함되어 있어, 4등급으로 판단하였습니다.
